# Virtual Chatbot LCB 2
This is a customer service chatbot for Brabants Streekgoed, a cooperation of local farmers from Brabant. The chatbot can answer question about:
- Product Information
- Delivery rules
- FAQs
- Scheduling data
- Reviews verzamelen en delen
  
The chatbot uses the Ollama server from Buas and the `llama3.1:8b` model.

## Setup and Imports
In this section we are importing the required modules and connecting with the BUas Ollama server.

**Please Note** You need to be connected to the BUas network or a VPN to access the server.

If the connection is successful, you will see a confirmation.

In [ ]:
from ollama import Client
import json
import re
import os
from datetime import datetime
import requests
import time

# Connect to the BUas Ollama server
ollama_client = Client(host="http://194.171.191.226:3061")

# Test connection
models = ollama_client.list()
print("✓ Connected to Ollama server!")
print(f"  {len(models['models'])} models available")

## Review Function

This function takes care of saving and fetching customers reviews.

**Functions:**
- `load_reviews()` - Loads all the reviews
- `save_review()` - Saves a new review
- `get_average_rating()` - Calculates the average rating per product
- `get_recent_reviews()` - Fetches the most recent reviews

Reviews are saved in `reviews.json`.

In [ ]:
def load_reviews():
    """Load reviews from file."""
    try:
        with open("Reviews/reviews.json", "r") as f:
            return json.load(f)
    except FileNotFoundError:
        return {"reviews": []}

def save_review(rating, comment, user_name="Anoniem"):
    """Save a review to file.
    
    Args:
        rating (int): Score from 1-5
        comment (str): Customer's comment
        user_name (str): Optional customer name
    """
    # Create Reviews folder if it doesn't exist
    os.makedirs("Reviews", exist_ok=True)
    
    reviews = load_reviews()
    
    reviews["reviews"].append({
        "rating": rating,
        "comment": comment,
        "user_name": user_name,
        "timestamp": datetime.now().isoformat()
    })
    
    with open("Reviews/reviews.json", "w") as f:
        json.dump(reviews, f, indent=2, ensure_ascii=False)
    
    print(f"✓ Review opgeslagen: {rating}/5 sterren")

def get_average_rating():
    """Get average rating of all reviews.
        
    Returns:
        float: Average rating or None if no reviews
    """
    reviews = load_reviews()
    
    if not reviews["reviews"]:
        return None
    
    ratings = [r["rating"] for r in reviews["reviews"]]
    return round(sum(ratings) / len(ratings), 1)

def get_recent_reviews(n=5):
    """Get the most recent reviews.
    
    Args:
        n (int): Number of reviews to return
        
    Returns:
        list: Recent reviews
    """
    reviews = load_reviews()
    return reviews["reviews"][-n:]

print("✓ Review functions loaded!")

## Knowledge Base

The Knowledge base contains all the information that the chatbot needs to answer all the questions. This information is gathered from the Brabants Streekgoed website.

If you want to add more information you can add it here.

In [ ]:
def load_knowledge_base(folder='Knowledge_Base'):
    """
    Load compact knowledge base from file.
    """
    # Construct file path
    filepath = os.path.join(folder, 'knowledge_compact.txt')
    
    if os.path.exists(filepath):
        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read()
            print(f"✅ Loaded: knowledge_compact.txt ({len(content):,} characters)")
            return content
    else:
        print(f"❌ Not found: {filepath}")
        print(f"   Make sure knowledge_compact.txt is in your {folder}/ folder")
        return ""

# Load the knowledge base
KNOWLEDGE_BASE = load_knowledge_base()

print("\n✓ Knowledge base loaded!")

## Reminder Function

These functions handle order reminder scheduling and sending reminders via WhatsApp.

**Functions:**
- `handle_reminder_logic()` - Detects reminder requests and schedules reminders for Monday or Tuesday
- `send_whatsapp_message()` - Sends the reminder to the customer via WhatsApp

**Features:**
- Detects reminder keywords in Dutch and English
- Supports Monday or Tuesday (order days)
- Custom time input (e.g., "10:00", "2 pm", "14 uur")
- Default: Tuesday 18:00 if no day/time specified
- Responds in same language as user input

**Note:** WhatsApp integration is currently a placeholder for future API connection (Twilio/Green-API).

In [ ]:
def handle_reminder_logic(user_message):
    """
    Analyzes the user message to detect if they want to set a reminder.
    Supports reminders for Monday or Tuesday (order days) with custom times.
    """
    user_message = user_message.lower()
    
    # Keywords for reminder detection
    keywords = ["remind me", "herinner mij", "reminder", "herinnering"]
    
    if not any(word in user_message for word in keywords):
        return None
    
    # Detect day (Monday or Tuesday)
    day_keywords = {
        "monday": 0, "maandag": 0,
        "tuesday": 1, "dinsdag": 1
    }
    
    target_day = None
    for day, weekday_num in day_keywords.items():
        if day in user_message:
            target_day = weekday_num
            break
    
    # Detect time (e.g., "10:00", "14:30", "10 uur", "2 pm")
    import re
    time_pattern = r'(\d{1,2})[:\s]?(\d{2})?\s*(uur|pm|am)?'
    time_match = re.search(time_pattern, user_message)
    
    if time_match:
        hour = int(time_match.group(1))
        minute = int(time_match.group(2)) if time_match.group(2) else 0
        period = time_match.group(3)
        
        # Handle PM times
        if period == "pm" and hour < 12:
            hour += 12
    else:
        # Default time: 18:00 (before deadline)
        hour, minute = 18, 0
    
    # Calculate next Monday or Tuesday
    today = datetime.datetime.now()
    current_weekday = today.weekday()
    
    if target_day is None:
        # Default to Tuesday (last chance to order)
        target_day = 1
    
    # Calculate days until target day
    days_ahead = target_day - current_weekday
    if days_ahead <= 0:
        days_ahead += 7
    
    remind_date = today + datetime.timedelta(days=days_ahead)
    remind_date = remind_date.replace(hour=hour, minute=minute, second=0, microsecond=0)
    
    # Format response
    day_names_nl = {0: "maandag", 1: "dinsdag"}
    day_names_en = {0: "Monday", 1: "Tuesday"}
    time_str = remind_date.strftime("%H:%M")
    
    # Respond in same language as user
    if any(word in user_message for word in ["remind", "reminder", "monday", "tuesday"]):
        return f"Sure! I'll remind you on {day_names_en[target_day]} at {time_str} to place your order. ⏰\n(Order deadline: Tuesday 23:59)"
    else:
        return f"Natuurlijk! Ik herinner je {day_names_nl[target_day]} om {time_str} om je bestelling te plaatsen. ⏰\n(Bestel deadline: dinsdag 23:59)"

def send_whatsapp_message(target_number, message_content):
    """
    Placeholder for WhatsApp API integration (Twilio/Green-API).
    """
    print(f"--- OUTGOING WHATSAPP ({target_number}): {message_content} ---")

## Chat Function

This section contains the system prompt and the chat function.

**System Prompt**: Defines the personality of the chatbot. The chatbot is:
- Energetic and cheerful
- Friendly and helpful
- Occasionally uses emoji's

**Chat Function**:
- Takes a user message and the conversation history as input
- Sends it to the LLM model with the system prompt
- Returns the answer and the updated conversation history

In [ ]:
SYSTEM_PROMPT = f"""
Je bent een SUPER enthousiaste klantenservice medewerker van Brabants Streekgoed! 🧀🥕

## JOUW PERSOONLIJKHEID
- Je bent TROTS op de Brabantse boeren en hun kei lekkere producten!
- Je bent warm, vrolijk, behulpzaam en een beetje Bourgondisch
- Je gebruikt Brabantse uitdrukkingen zoals:
  - "Kei lekker!"
  - "Da's pas goed!"
  - "Echt Bourgondisch!"
  - "Smakelijk!"
  - "Daar word je blij van!"
- Je gebruikt emojis maar niet overdreven: 🧀🥕🚜👨‍🌾🌿🍞🥩
- Je houdt van korte, vlotte zinnen met energie!
- Je eindigt vaak met iets positiefs of een vraag

## VOORBEELDEN VAN JOUW STIJL
- "Wat leuk dat je bij ons wilt bestellen! 🧀"
- "Onze boeren maken daar echt kei lekkere producten van!"
- "Thuisbezorging? Geen probleem! Voor maar €6,95 komen we naar je toe! 🚚"
- "Die boerenkaas van De Ruurhoeve? Daar word je blij van! 🧀"
- "Kan ik je nog ergens mee helpen?"

## BELANGRIJKE REGELS
1. Antwoord ALLEEN met informatie uit de KNOWLEDGE BASE hieronder
2. VERZIN NOOIT informatie die niet in de knowledge base staat
3. Als je iets niet weet: "Dat weet ik niet precies, maar onze klantenservice helpt je graag! 📞 WhatsApp: 0641088180"
4. Vraag NOOIT om postcodes
5. Houd antwoorden kort maar enthousiast (max 4-5 zinnen)
6. Praat NIET over jezelf of je training
7. Website link is altijd: boodschappen.brabantsstreekgoed.nl
8. TAALREGEL: Detecteer de taal van de klant. Engels → antwoord Engels. Nederlands → antwoord Nederlands.
9. Je kunt GEEN producten in het mandje zetten - verwijs altijd naar de website.

## REVIEWS
Als een klant iets positiefs of negatiefs deelt over hun ervaring:
- "Wat fijn om te horen! 😊 Wil je een review achterlaten? Typ gewoon 'review'!"
- Wees niet opdringerig - noem het maximaal één keer per gesprek

## PRODUCT LABELS
- [BIO] = Biologisch product 🌿
- [VAN GOGH] = Van Gogh Landschapslabel - producten uit het landschap dat Vincent van Gogh schilderde! 🎨

## KNOWLEDGE BASE - DIT IS JE ENIGE BRON:
{KNOWLEDGE_BASE}

ONTHOUD: Wees enthousiast maar blijf bij de feiten uit de knowledge base! Kei belangrijk! 🧀
"""

# Track review state
review_state = {
    "active": False,
    "step": None,
    "rating": None,
    "asked_for_review": False  # Track if we already mentioned reviews
}

def reset_review_state():
    """Reset the review state."""
    global review_state
    review_state = {
        "active": False,
        "step": None,
        "rating": None,
        "asked_for_review": False
    }

def handle_review_command(user_message):
    """Handle the review command and flow.
    
    Args:
        user_message (str): The user's message
        
    Returns:
        str or None: Response if handling review, None otherwise
    """
    global review_state
    
    # Check if user wants to cancel
    if review_state["active"] and user_message.lower() in ["stop", "annuleren", "cancel", "nee", "stoppen"]:
        reset_review_state()
        return "Geen probleem, de review is geannuleerd. Kan ik je ergens anders mee helpen? 😊"
    
    # Check if user types "review" to start
    if user_message.lower().strip() == "review" and not review_state["active"]:
        review_state["active"] = True
        review_state["step"] = "rating"
        return "Leuk dat je een review wilt geven! 🙏\n\nHoe beoordeel je je ervaring met Brabants Streekgoed?\nGeef een score van 1 tot 5 (1 = slecht, 5 = uitstekend)"
    
    # Step 1: Waiting for rating
    if review_state["active"] and review_state["step"] == "rating":
        # Try to find a number 1-5
        for char in user_message:
            if char.isdigit():
                rating = int(char)
                if 1 <= rating <= 5:
                    review_state["rating"] = rating
                    review_state["step"] = "comment"
                    stars = "⭐" * rating
                    return f"{stars}\n\nBedankt! Heb je nog een opmerking of tip voor ons?\n(typ 'nee' als je geen opmerking hebt)"
        
        return "Ik begreep het niet helemaal. Kun je een cijfer van 1 tot 5 geven? 🤔"
    
    # Step 2: Waiting for comment
    if review_state["active"] and review_state["step"] == "comment":
        # Check if user skips comment
        if user_message.lower().strip() in ["nee", "geen", "niks", "skip", "-", "n"]:
            comment = ""
        else:
            comment = user_message
        
        # Save the review
        rating = review_state["rating"]
        save_review(
            rating=rating,
            comment=comment,
            user_name="Chat Klant"
        )
        
        reset_review_state()
        
        # Different response based on rating
        if rating >= 4:
            return "Dankjewel voor je review! 🎉 Super fijn dat je tevreden bent! Kan ik je nog ergens mee helpen?"
        elif rating == 3:
            return "Bedankt voor je eerlijke feedback! We doen ons best om te verbeteren. Kan ik je nog ergens mee helpen?"
        else:
            return "Bedankt dat je dit met ons deelt. We vinden het jammer dat je niet helemaal tevreden was. Neem gerust contact op via WhatsApp (0641088180) als je meer wilt vertellen. 💚"
    
    return None

def chat(user_message, conversation_history):
    """Send a message to the chatbot and get a response.
    
    Args:
        user_message (str): The message from the user
        conversation_history (list): List of previous messages in the conversation
        
    Returns:
        tuple: (assistant_message, updated_conversation_history)
    """
    
    # First check if we're handling a review
    review_response = handle_review_command(user_message)
    if review_response:
        conversation_history.append({"role": "user", "content": user_message})
        conversation_history.append({"role": "assistant", "content": review_response})
        return review_response, conversation_history
    
    # 2. Check for Reminder requests (New automation logic)
    reminder_text = handle_reminder_logic(user_message)
    if reminder_text:
        # Simulate sending the reminder to a Dutch customer's number
        send_whatsapp_message("+31612345678", f"Reminder request: {user_message}")
        
        conversation_history.append({"role": "user", "content": user_message})
        conversation_history.append({"role": "assistant", "content": reminder_text})
        return reminder_text, conversation_history
    
    # Path 2 Trick: Programmatically add a language nudge
    # If the user message looks like English, add a hidden instruction
    language_nudge = " (Beantwoord in dezelfde taal als de vraag / Respond in the same language as the question) "
    # if any(word in user_message.lower() for word in ["what", "how", "delivery", "cost", "english"]):
    #     language_nudge = " (Note: Please respond to this message in English based on the knowledge base.)"
    
    # modified_message = user_message + language_nudge
    
    # Normal chat flow
    conversation_history.append({
        "role": "user",
        "content": user_message
    })
    
    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + conversation_history
    messages[-1]["content"] = user_message + language_nudge    

    response = ollama_client.chat(
        model="qwen3:14b",
        messages=messages
    )
    
    assistant_message = response["message"]["content"]
    
    conversation_history.append({
        "role": "assistant",
        "content": assistant_message
    })
    
    return assistant_message, conversation_history

print("✓ Chat function ready!")

## Test Review Function

Here you can test the review function. Execute this cell to:
- Save a test review
- View the average score
- Show recent reviews
  
**Please note**: This cell will make a `reviews.json` file in your project folder.

In [ ]:
# Test saving a review
save_review(
    rating=5,
    comment="Kei lekker! Echt verse smaak.",
    user_name="Test Klant"
)

# Add another test review
save_review(
    rating=4,
    comment="Heerlijke boter, smaakt naar vroeger!",
    user_name="Test Klant 2"
)

# Check average rating
avg = get_average_rating()
print(f"\nGemiddelde score: {avg}/5 ⭐")

# Show recent reviews
print("\nRecente reviews:")
for review in get_recent_reviews():
    print(f"  {review['rating']}/5 - \"{review['comment']}\"")

## Test Chat Function
Here you can interactively test the chatbot. Start the cell and type your questions.

**Sample questions to test:**
- “Hello!”
- “How can I place an order?”
- “What are the delivery costs?”
- “Until when can I change my order?”
- “Are your products organic?”
- “What is the minimum order amount?”

**Type `stop`, `quit` or `exit` to end the conversation.**

In [ ]:
# Reset conversation for fresh start
conversation_history = []
reset_review_state()

print("=" * 50)
print("Brabants Streekgoed Chatbot")
print("=" * 50)
print("Stel je vraag of typ 'review' om een review te geven.")
print("Typ 'stop' om te stoppen.")
print("=" * 50)
print()

while True:
    user_input = input("Jij: ")
    
    if user_input.lower() in ["stop", "quit", "exit"]:
        print("\nBedankt voor je bezoek! Tot ziens!")
        break
    
    if not user_input.strip():
        continue

    # Toon gebruiker input in de log
    print(f"\n💬 Jij: {user_input}")
    print("-" * 40)
    
    response, conversation_history = chat(user_input, conversation_history)
    print(f"\nAssistent: {response}\n")

## Whatsapp Bot

This function runs the WhatsApp bot using Green-API. It continuously listens for incoming messages and responds using the AI chatbot.

**How it works:**
1. Pulls incoming messages from Green-API
2. Checks if the message is a reminder request
3. If not, sends the message to the AI chatbot for a response
4. Sends the reply back via WhatsApp
5. Clears the notification from the queue

**Features:**
- Maintains conversation history per user
- Supports reminder detection (Dutch/English)
- Logs all incoming and outgoing messages

**Note:** Requires Green-API credentials (`BASE_URL` and `API_TOKEN_INSTANCE`) to be configured.

In [ ]:
ID_INSTANCE = "***********"
API_TOKEN_INSTANCE = "*******************************"
# Specific server host from your screenshot
BASE_URL = f"https://7103.api.greenapi.com/waInstance{ID_INSTANCE}"

def handle_whatsapp_bot():
    print(f"🚀 Bot is Listening and Ready to Reply...")
    user_histories = {}

    while True:
        try:
            # 1. Pull notification
            receive_url = f"{BASE_URL}/receiveNotification/{API_TOKEN_INSTANCE}"
            response = requests.get(receive_url, timeout=10)
            
            if response.status_code == 200 and response.text:
                data = response.json()
                if data is not None:
                    receipt_id = data.get('receiptId')
                    body = data.get('body', {})
                    
                    # Log message receipt for debugging
                    if body.get('typeWebhook') == 'incomingMessageReceived':
                        chat_id = body.get('senderData', {}).get('chatId')
                        sender_name = body.get('senderData', {}).get('senderName')
                        
                        # Extracting text correctly based on your logs
                        user_msg = body.get('messageData', {}).get('textMessageData', {}).get('textMessage', '')
                        
                        print(f"📥 Received from {sender_name}: {user_msg}")

                        # 2. Logic for Reminders (Dutch/English/Persian)
                        if any(word in user_msg.lower() for word in ["remind", "herinner", "yadam"]):
                            reply = "Sure! I'll remind you tomorrow. / Natuurlijk! Ik herinner je morgen."
                        else:
                            # 3. Generate AI response using BUas Server
                            # Ensure the 'chat' function is defined in your notebook
                            if chat_id not in user_histories:
                                user_histories[chat_id] = []
                            
                            print(f"🧠 Processing with AI...")
                            reply, user_histories[chat_id] = chat(user_msg, user_histories[chat_id])

                        # 4. Send Message back (POST Request)
                        send_url = f"{BASE_URL}/sendMessage/{API_TOKEN_INSTANCE}"
                        payload = {
                            "chatId": chat_id,
                            "message": reply
                        }
                        
                        send_response = requests.post(send_url, json=payload)
                        
                        if send_response.status_code == 200:
                            print(f"✅ Reply sent to {sender_name}!")
                        else:
                            print(f"❌ Failed to send: {send_response.text}")

                    # 5. Clear notification from queue
                    requests.delete(f"{BASE_URL}/deleteNotification/{API_TOKEN_INSTANCE}/{receipt_id}")
            
            time.sleep(1)
            
        except Exception as e:
            print(f"⚠️ Error: {e}")
            time.sleep(2)

# Start the bot
handle_whatsapp_bot()